In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
import pandas as pd

In [36]:


# Laden der Bevolkerung.csv 
bevo = pd.read_excel('bevolkerung.xls',)


# Umbenennenung der Spalte
bevo = bevo.rename(columns={'Country Code': 'ISO3'})

# Reduziere auf nur ISO3 + population
bevo = bevo[['ISO3', '2023']]

# Laden der merged.csv
merged = pd.read_csv('merged.csv')

# Führe den Merge anhand von ISO3 durch
merged = merged.merge(bevo, on='ISO3', how='left')

# Umbennung der Spalte
merged = merged.rename(columns={'2023': 'Population'})
merged = merged.rename(columns={'RP Financing $': 'RP', 'FA Financing $':'FA'})



In [37]:
#mergen der Länderklassifikation

df_class = pd.read_excel("CLASS.xlsx")

# Reduziere auf nur ISO3 + population
df_class = df_class[['ISO3', 'Income group']]
# Führe den Merge anhand von ISO3 durch
merged = merged.merge(df_class, on='ISO3', how='left')


# Speichere in merged.csv (überschreibt die Datei)
merged.to_csv('mergedpop.csv', index=False)

In [38]:
# Mapping der Einkommensgruppen zu numerischen Codes
income_mapping = {
    'Low income': 1,
    'Lower middle income': 2,
    'Upper middle income': 3,
    'High income': 4
}

# Neue Spalte 'Income Group Code' erstellen
merged['Income Group Code'] = merged['Income group'].map(income_mapping)

In [39]:
print(merged)


     Unnamed: 0                           Countries ISO3  \
0             0                         Afghanistan  AFG   
1             1                             Albania  ALB   
2             2                             Algeria  DZA   
3             3                              Angola  AGO   
4             4                           Argentina  ARG   
..          ...                                 ...  ...   
110         110  Venezuela (Bolivarian Republic of)  VEN   
111         111                            Viet Nam  VNM   
112         112                               Yemen  YEM   
113         113                              Zambia  ZMB   
114         114                            Zimbabwe  ZWE   

                              Region   SIDS   LDCs  # RP  # FA          RP  \
0                       Asia-Pacific  False   True     3     1  1599987.00   
1                     Eastern Europe  False  False     6     5  5440619.00   
2                             Africa  False  

In [40]:
#Finanzierungsgelder pro Einwohner

merged["RPPOP"] = merged["RP"] / merged["Population"]
merged["FAPOP"] = merged["FA"] / merged["Population"]


In [41]:
# Relevante Spalten, die mit 0 gefüllt werden sollen
# Weil wenn die Spalte leer ist, dann gab es keine Gelder, also nicht NA sondern einfach 0
fill_cols = ['RP', 'FA', 'RPPOP', 'FAPOP', ]

# Nur falls die Spalten auch vorhanden sind
for col in fill_cols:
    if col in merged.columns:
        merged[col] = merged[col].fillna(0)


print("Fehlende Werte in RP/FA/RPPOP/FAPOP wurden mit 0 ersetzt.")


Fehlende Werte in RP/FA/RPPOP/FAPOP wurden mit 0 ersetzt.


In [42]:
# GDP per Capita hinzufügen
# einlesen
df_gdp = pd.read_excel("gdppercapita.xls")
print(df_gdp)

# Jahre überprüfen in abfolgender Reihenfolge, sprich das aktuellste zuerst (bis 1999)
years = [str(year) for year in range(2023, 1999, -1)]

# Neue Spalte erstellen und mit NAN initialisieren
df_gdp['GDPCap'] = np.nan

# Durch die Spalten gehen und den aktuellsten Wert nehmen
for year in years:
    if year in df_gdp.columns:
        df_gdp['GDPCap'] = df_gdp['GDPCap'].fillna(df_gdp[year])

# Überprüfung der Länder welche Nan-Werte haben
countries_with_nan_gdpc = df_gdp[df_gdp['GDPCap'].isna()]['ISO3'].tolist()
columns2=['ISO3', 'GDPCap']

# Mergen

merged = merged.merge(df_gdp[columns2], on='ISO3', how='left')

merged['LogGDPCap'] = np.log(merged['GDPCap'])
print("Merged DataFrame:")
print(merged.head())
print("\nCountries with NaN in GDPCap:")
print(countries_with_nan_gdpc)


merged.to_csv('NEWmergedpop.csv', index=False)

                    Country Name ISO3  1960  1961  1962  1963  1964  1965  \
0                          Aruba  ABW   NaN   NaN   NaN   NaN   NaN   NaN   
1    Africa Eastern and Southern  AFE   NaN   NaN   NaN   NaN   NaN   NaN   
2                    Afghanistan  AFG   NaN   NaN   NaN   NaN   NaN   NaN   
3     Africa Western and Central  AFW   NaN   NaN   NaN   NaN   NaN   NaN   
4                         Angola  AGO   NaN   NaN   NaN   NaN   NaN   NaN   
..                           ...  ...   ...   ...   ...   ...   ...   ...   
261                       Kosovo  XKX   NaN   NaN   NaN   NaN   NaN   NaN   
262                  Yemen, Rep.  YEM   NaN   NaN   NaN   NaN   NaN   NaN   
263                 South Africa  ZAF   NaN   NaN   NaN   NaN   NaN   NaN   
264                       Zambia  ZMB   NaN   NaN   NaN   NaN   NaN   NaN   
265                     Zimbabwe  ZWE   NaN   NaN   NaN   NaN   NaN   NaN   

     1966  1967  ...          2015          2016          2017          201

In [44]:
#NDGAIN  hinzufügen

#einlesen
df_gain = pd.read_csv("gain.csv")
print(df_gain)

# Umbennenung
df_gain.rename(columns={'2022': 'NDGAIN'}, inplace=True)
columns3=['ISO3', 'NDGAIN']

# Mergen
merged = merged.merge(df_gain[columns3], on='ISO3', how='left')
merged.to_csv('NEWmergedpop.csv', index=False)

    ISO3                              Name       1995       1996       1997  \
0    AFG                       Afghanistan  34.782309  34.775913  34.855036   
1    ALB                           Albania  43.676623  43.659503  43.596649   
2    DZA                           Algeria  45.112792  45.225430  45.122920   
3    AND                           Andorra        NaN        NaN        NaN   
4    AGO                            Angola  34.723153  34.712671  34.656007   
..   ...                               ...        ...        ...        ...   
187  VEN  Venezuela, Bolivarian Republic o  45.753557  45.688648  45.789780   
188  VNM                          Viet Nam  41.893196  42.006936  42.046623   
189  YEM                             Yemen  37.229012  37.298996  37.217630   
190  ZMB                            Zambia  40.634959  40.616142  40.764332   
191  ZWE                          Zimbabwe  38.438700  38.225003  38.056483   

          1998       1999       2000       2001    

In [45]:
#WGI-Indikatoren

# Datei laden
wgi = pd.read_excel("wgidataset.xlsx")

# Nur die 5 relevanten Indikatoren auswählen
relevant_indicators = ["va", "pv", "ge", "rl", "cc"]
wgi = wgi[wgi["indicator"].isin(relevant_indicators)]

# Sicherstellen, dass 'estimate' numerisch ist
wgi["estimate"] = pd.to_numeric(wgi["estimate"], errors="coerce")

# Sortieren nach Jahr absteigend, damit neueste Werte oben stehen
wgi = wgi.sort_values(by=["code", "indicator", "year"], ascending=[True, True, False])

# Entferne Duplikate: Behalte pro (Land, Indikator) nur den neuesten Eintrag
wgi_latest = wgi.drop_duplicates(subset=["code", "indicator"], keep="first")

# Pivotieren: ISO3 ("code") = Zeilen, Indikatoren = Spalten
wgi_wide = wgi_latest.pivot(index="code", columns="indicator", values="estimate")


# Index zurücksetzen für Merge
wgi_wide = wgi_wide.reset_index()

# Mergen 
merged = merged.merge(wgi_wide, left_on="ISO3", right_on="code", how="left")

# Zwischenspeichern
merged.to_csv('NEWmergedpop.csv', index=False)


In [46]:
# Hinzufügen Faktor Ambition
df_faktor = pd.read_csv('faktorndc_explorer.csv')


# Umbenenne der Spalte
df_faktor = df_faktor.rename(columns={'country_code': 'ISO3'})



# Führe den Merge anhand von ISO3 durch
merged = merged.merge(df_faktor, on='ISO3', how='left')

# Zwischenspeichern
merged.to_csv('NEWmergedpop.csv', index=False)

In [47]:
# Hinzufügen vom Faktor Delta CO2/BIP 
df_delta = pd.read_csv('co2intensitydelta.csv')


# Umbenenne der Spalte
df_delta = df_delta.rename(columns={'Code': 'ISO3'})

# Nur ISO3 und Delta-Spalte auswählen
df_delta = df_delta[['ISO3', 'δCO2/GDP']]
# Führe den Merge anhand von ISO3 durch
merged = merged.merge(df_delta, on='ISO3', how='left')

#Zwischenspeichern
merged.to_csv('NEWmergedpop.csv', index=False)

In [1]:
#HKA erstellen

# CSV-Datei laden
df = pd.read_csv("NEWmergedpop.csv")

# Governance-Variablen definieren
governance_vars = ['va', 'pv', 'ge', 'rl', 'cc']

# Governance-Daten extrahieren und fehlende Werte entfernen
df_gov = df[governance_vars].dropna()

# Standardisieren
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_gov)

# PCA mit 2 Komponenten
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Neue PCA-Variablen ins Haupt-DataFrame einfügen
df.loc[df_gov.index, 'Gov_PCA1'] = X_pca[:, 0]
df.loc[df_gov.index, 'Gov_PCA2'] = X_pca[:, 1]


# Finale Speicherung des Datasets
df.to_csv("NEWmergedpop.csv", index=False)
print("✅ Datei 'NEWmergedpop.csv' mit Gov_PCA1 und Gov_PCA2 gespeichert.")


✅ Datei 'NEWmergedpop.csv' mit Gov_PCA1 und Gov_PCA2 gespeichert.
